In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

content = []
with open("EE_summary_hg38.tsv") as f:
    for line in f: 
        content.append(line.strip().split("\t"))

df = pd.DataFrame(content)

#Replace NA values 
df.replace('NA', "0", inplace=True)

#make numeric values of interest and split values 
def convert_to_int(x):
    return [int(value) for value in x]

df[8] = df[8].str.split(',')
df[9] = df[9].str.split(',').apply(convert_to_int)
df[10] = df[10].str.split(',')
df[11] = df[11].str.split(',').apply(convert_to_int)
df[20] = df[20].str.split(',')
df[21] = df[21].str.split(',').apply(convert_to_int)
df[23] = df[23].str.split(',')
df[24] = df[24].str.split(',').apply(convert_to_int)
df[27] = df[27].str.split(',')
df[28] = df[28].str.split(',').apply(convert_to_int)
df[6] = df[6].astype(int)
df[7] = df[7].astype(int)
df[16] = df[16].astype(int)
df[17] = df[17].str.replace(',', '.') #in python we need to use a dot for float numbers
df[18] = df[18].str.replace(',', '.')
df[19] = df[19].str.replace(',', '.')
df[17] = df[17].astype(float)
df[18] = df[18].astype(float)
df[19] = df[19].astype(float)
df[22] = df[22].astype(int)
df[26] = df[26].astype(int)

In [ ]:
#starr overlap
dic_starr={}
with open("/mnt/project/exonhancer/ZENODO_REPO/STARR-seq_catalog/overlap/hg38_ee.tsv") as file:
    for line in file:           
        if line.strip().split()[3] not in dic_starr:
            dic_starr[line.strip().split()[3]] = []
        else:
            cell = line.strip().split()[9].split(".")[1]
            dic_starr[line.strip().split()[3]].append(cell)


dic_starr_silent={}
with open("/mnt/project/exonhancer/ZENODO_REPO/STARR-seq_catalog/overlap/hg38_ee_silent_peaks.tsv") as file:
    for line in file:           
        if line.strip().split()[3] not in dic_starr_silent:
            dic_starr_silent[line.strip().split()[3]] = []
        else:
            cell = line.strip().split()[9].split(".")[1]
            dic_starr_silent[line.strip().split()[3]].append(cell)

with open("EE_summary_hg38.tsv") as file:
    for line in file:           
        if line.strip().split()[3] not in dic_starr:
            dic_starr[line.strip().split()[3]] = []
        if line.strip().split()[3] not in dic_starr_silent:
            dic_starr_silent[line.strip().split()[3]] = []

In [ ]:
#remap peaks / dnase / starrseq / size
def val_peak_a549(row):
    val = row[10].index("A-549")
    return row[11][val]

def val_peak_gm(row):
    val = row[10].index("GM12878")
    return row[11][val]

In [ ]:
#All K-562
df_k562_gold = df[df.apply(lambda row: ('K-562' in row[10]) and ('K-562' in row[20]) and (row[6] >= 130), axis=1)]
k562_g = df_k562_gold.pop(3).to_list()

df_k562_silver = df[df.apply(lambda row: ('K-562' in row[10]) and (len(dic_starr[row[3]]) > 0) and (row[6] >= 130) and ('K-562' not in dic_starr_silent[row[3]]), axis=1)]
k562_s = []
for index, row in df_k562_silver.iterrows():
    val = row[10].index("K-562")
    if row[11][val] >= 3:
        k562_s.append(row[3]) 

print(len(set(k562_g + k562_s)))

2467


In [85]:
k2000 = set(k562_g + k562_s)
with open("/home/mouren/Data/valid_exp_starr/list_raw_v3/K562_id", 'w') as file:
    for item in k2000:
        file.write(str(item) + '\n')

In [ ]:
#All A-549
df_a549_gold = df[df.apply(lambda row: ('A-549' in row[10]) and ('A-549' in row[20]) and (row[6] >= 130), axis=1)]
a549_g = df_a549_gold.pop(3).to_list()

df_a549_silver = df[df.apply(lambda row: ('A-549' in row[10]) and (len(dic_starr[row[3]]) > 0) and (row[6] >= 130) and ('A-549' not in dic_starr_silent[row[3]]), axis=1)]
a549_s = []
for index, row in df_a549_silver.iterrows():
    val = row[10].index("A-549")
    if row[11][val] >= 3:
        a549_s.append(row[3])

print(len(set(a549_g + a549_s)))

807


In [87]:
a = set(a549_g + a549_s)
with open("/home/mouren/Data/valid_exp_starr/list_raw_v3/A549_id", 'w') as file:
    for item in a:
        file.write(str(item) + '\n')

In [ ]:
#All GM possible 
df_gm_gold = df[df.apply(lambda row: ('GM12878' in row[10]) and ('GM12878' in row[20]) and (row[6] >= 130), axis=1)]
gm_g = df_gm_gold.pop(3).to_list()

df_gm_silver = df[df.apply(lambda row: ('GM12878' in row[10]) and (len(dic_starr[row[3]]) > 0) and (row[6] >= 130) and ('GM12878' not in dic_starr_silent[row[3]]), axis=1)]
gm_s = []
for index, row in df_gm_silver.iterrows():
    val = row[10].index("GM12878")
    if row[11][val] >= 3:
        gm_s.append(row[3])

print(len(set(gm_g + gm_s)))

867


In [90]:
g = set(gm_g + gm_s)
with open("/home/mouren/Data/valid_exp_starr/list_raw_v3/GM12878_id", 'w') as file:
    for item in g:
        file.write(str(item) + '\n')

In [95]:
print(len(set(k562_g + k562_s + a549_g + a549_s + gm_g + gm_s)))

3144


In [94]:
other = set(a549_g + a549_s + gm_g + gm_s)

result = other - k2000  # This will print the elements that are in other but not in k2000
print(len(result))
with open("/home/mouren/Data/valid_exp_starr/list_raw_v3/A549_GM12878_Uniques_id", 'w') as file:
    for item in result:
        file.write(str(item) + '\n')

677
